In [44]:
from pydantic import BaseModel, Field;
from agents import Agent, trace, Runner, function_tool, WebSearchTool
from agents.model_settings import ModelSettings
import sendgrid
import os 
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown
import asyncio

In [45]:
HOW_MANY_SEARCHES = 1

INSTRUCTIONS = f"""You are a helpful research assistance. When you are given with a query your job is to come up with a set of web searches that will
extract the most revelant information from the web related to the query. Provide {HOW_MANY_SEARCHES} web searches the best describe the query.
"""

In [46]:
class WebSearchItem(BaseModel):
    web_search_term : str  = Field(description="This is the search term that is used to search the web")
    reason : str = Field(description="Your reason why the search term is relevant")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to best answer the query")

In [47]:
planner_agent = Agent(
    name= "Planner Agent",
    instructions=INSTRUCTIONS,
    output_type=WebSearchPlan,
    model="gpt-4o-mini",
)

In [48]:
message = "Latest Ai frameworks in 2025"

In [49]:
@function_tool
def send_email(subject:str, html_body:str) -> Dict[str,str]:
    sg = sendgrid.SendGridAPIClient(api_key= os.environ.get('SENDGRID_API_KEY'))
    to_email = To('kasunkalharaweather@gmail.com')
    from_email = Email('pharshana719@gmail.com')
    content = Content("text/html",html_body)
    mail = Mail(from_email,to_email, subject,content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status":"success"}



In [50]:
email_agent_instructions = f"""You are able to send nicely formatted HTML email based on a detailed report
You will be provided with a detailed report. You shoud use the email sending tool that is provided and send one email providing the 
report converted into clean, well presented HTML with an appropriate subject line.
"""

email_agent = Agent(
    name="email agent",
    tools=[send_email],
    model="gpt-4o-mini",
    instructions=email_agent_instructions,
  
)

In [51]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."


search_agent = Agent(
    name="web search agent",
    model="gpt-4o-mini",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model_settings=ModelSettings(tool_choice="required")


)

In [52]:

writer_agent_instructions = f"""You are a senior researcher tasked withy writing a cohesive research report for a research query
You will be provided with the original query, and some initial reseaerch done by a research assistant\n 
You should first come up with an outline for the report that describes the structure and flow of the report. Then, generate the report and return that as your final output.\n
The final output should be in markdown format, and it should be lengthy and detailed. Aim for 5-10 pages of content, at least 1000 words.
"""

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


report_writer_agent = Agent(
    name="report writing agent",
    instructions=writer_agent_instructions,
    model="gpt-4o-mini",
    output_type=ReportData,

)


In [53]:
async def plan_search(query:str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planing Searches")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print("Will perform  {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan : WebSearchPlan):
    
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    print("Finished searching")
    results = asyncio.gather(*tasks)
    return results


async def search(search_item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term : {search_item.web_search_term}\n Reason for searching :{search_item.reason} "
    result = await Runner.run(search_agent, input)
    return result.final_output





In [54]:
async def write_report(query : str, search_results : list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"search query : {query}\n Summarised search results: {search_results}"
    result = await Runner.run(report_writer_agent,input)
    return result.final_output

async def send_email_func(report : ReportData):

    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report;
    

In [55]:
query = "Latest AI frameworks in 2025"

with trace("report_generation_and_send"):
    print("Starting Research")
    search_plan = await plan_search(query=query)
    search_results = await perform_searches(search_plan=search_plan)
    report = await write_report(query=query, search_results=search_results)
    result = await send_email_func(report=report)
    print("Done and Dusted");

    

Starting Research
Planing Searches
Will perform  {len(result.final_output.searches)} searches
Finished searching
Thinking about report...
Email sent
Done and Dusted
